In [ ]:
from pathlib import Path
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils
from hydromt.log import setuplog


## Basic Config

In [ ]:
project_root = Path().resolve()
model_root = project_root/"models"
model_name = "20m_final"

# model resolution in meters
res = 20

# mask file
mask_file = project_root/"data"/"mask"/"mask_25833.geojson"

# Initial water levels
ini_wl_file = project_root/"data"/"mask"/"zsini_regridded_20m_final.tif"

# topobathy datalib
datalib = project_root/"data"/"topobathy"/"data.yml"

# dike geometry
dike_file = project_root/"data"/"dikes"/"all_dikes.gpkg"

# waterlevel forcing point
x = [372368.1]
y = [6033653.6]

# Observation points for model validation
obs = {'Station': ["Barhoeft","Barth","Althagen"], 
     'geometry': [
         Point(371100.0, 6034300.0), 
         Point(352691.1, 6027443.5),
         Point(332296.2, 6027473.9),
         ]}



## Base SFINCS setup

In [ ]:
logger = setuplog(log_level=20)

sf = SfincsModel(
    root=model_root/model_name,
    data_libs=datalib,
    mode="w+",
    logger=logger
)

sf.setup_config(
    # timings will be overwritten later in scenario setup
    tref="20100101 000000",
    tstart="20100101 000000",
    tend="20100102 000000",
    # misc time options
    dtout=10400.0,
    dthisout=600.0,
    dtmaxout=259200.0,
    tspinup=86400,
    # zsini will be overwritten later in base setup
    zsini=-1,
)

# Create grid
sf.setup_grid_from_region(
    region={"geom": mask_file},
    res=res,
    crs="EPSG:25833", # local crs
    rotated=False
)

# setup topobathy
datasets_dep = [{"elevtn": "MVtopo_masked", "zmin": 0.01},{"elevtn": "ELC_INSPIRE"},{"elevtn": "merit_hydro"}]
dep = sf.setup_dep(datasets_dep=datasets_dep)

# setup active mask
sf.setup_mask_active(
    include_mask=mask_file,
    zmax=-50.0,
)
sf.setup_mask_bounds(
    btype="waterlevel",
    zmax=0.0,
)

# setup roughness
sf.setup_manning_roughness(
    manning_land=0.04,
    manning_sea=0.02,
    rgh_lev_land=0,  # the minimum elevation of the land
)

# setup initial water levels
ini_ds = xr.open_dataset(ini_wl_file, engine="rasterio")
ini_ds = ini_ds.rename({"band_data": "zsini"})
sf.set_states(data=ini_ds["zsini"])
sf.set_config("inifile", "sfincs.ini")

## Setup waterlevel forcing point and obs stations

In [ ]:
pnts = gpd.points_from_xy(x, y)
index = [1]  # NOTE that the index should start at one
bnd = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf.crs)

# Create some placeholder for waterlevel timeseries
# Will be overwritten in scenario setup
time = pd.date_range(
    start=utils.parse_datetime(sf.config['tstart']),
    end=utils.parse_datetime(sf.config['tend']),
    periods=3
)

bzs = [0.0, 1.0, 0.0]
bzspd = pd.DataFrame(index=time, data=bzs, columns=index)

sf.setup_waterlevel_forcing(timeseries=bzspd, locations=bnd, buffer=1000)

# Setup obs points
gdf = gpd.GeoDataFrame(obs, crs=sf.crs)
sf.setup_observation_points(
    locations=gdf, merge=False
)

## Setup dikes

In [ ]:
gdf_floodwall = sf.data_catalog.get_geodataframe(
    dike_file, geom=sf.region, crs=sf.crs
)

if (gdf_floodwall.geometry.type == "MultiLineString").any():
    gdf_floodwall = gdf_floodwall.explode()

gdf_floodwall["par1"] = 0.6

sf.setup_structures(
    structures=gdf_floodwall,
    stype="weir",
    merge=False,
)

## Save model

In [ ]:
sf.write()